### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/25397/Absolute_Duo/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,7):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/25397/Absolute_Duo/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  120 non-null    object
dtypes: object(1)
memory usage: 1.1+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n2\n\n\nStory\n2\n\n\nAnimatio...
1,\n\n\n\nOverall\n4\n\n\nStory\n8\n\n\nAnimatio...
2,\n\n\n\nOverall\n2\n\n\nStory\n2\n\n\nAnimatio...
3,\n\n\n\nOverall\n2\n\n\nStory\n0\n\n\nAnimatio...
4,\n\n\n\nOverall\n6\n\n\nStory\n5\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


 Overall 2 Story 2 Animation 2 Sound 2 Character 2 Enjoyment 2 Have you ever had trouble spotting a bottom-of-the-barrel title? Finding the product of studio algorithm, where the intent is solely that of a product and nothing more? Shows with as much integrity as a used car salesman with a tacky poker-dot tie, or as much validity as McDonald's promoting "health" through thinly cut apple slices and a bottle of processed milk? Hogwash that pretends to have artistic merit? And if so, would you like to sift through bottom-tier anime before so much as letting it occupy time on your media device? If you've answered yes to this, here are some quick tips to help you out: ▸Is there a typical school setting? ☑ ▸Is the protagonist a pussy magnet? ☑ ▸Is there a beach or hot spring episode? ☑ ▸Is there a love interest story that goes nowhere? ☑ ▸Is there unnecessary fanservice in the cover photo? ☑ ▸Is the lead easily flustered by anything involving females? ☑ ▸Does it go from tensionless fights to

In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'have', 'you', 'ever', 'had', 'trouble', 'spotting', 'bottom', 'of', 'the', 'barrel', 'title', 'finding', 'the', 'product', 'of', 'studio', 'algorithm', 'where', 'the', 'intent', 'is', 'solely', 'that', 'of', 'product', 'and', 'nothing', 'more', 'shows', 'with', 'as', 'much', 'integrity', 'as', 'used', 'car', 'salesman', 'with', 'tacky', 'poker', 'dot', 'tie', 'or', 'as', 'much', 'validity', 'as', 'mcdonald', 'promoting', 'health', 'through', 'thinly', 'cut', 'apple', 'slices', 'and', 'bottle', 'of', 'processed', 'milk', 'hogwash', 'that', 'pretends', 'to', 'have', 'artistic', 'merit', 'and', 'if', 'so', 'would', 'you', 'like', 'to', 'sift', 'through', 'bottom', 'tier', 'anime', 'before', 'so', 'much', 'as', 'letting', 'it', 'occupy', 'time', 'on', 'your', 'media', 'device', 'if', 'you', 've', 'answered', 'yes', 'to', 'this', 'here', 'are', 'some', 'quick', 'tips', 'to', 'help', 'you', 'out', 'is', 'there', 'typical',

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation_sound', 'character', 'enjoyment', 'absolute_duo', 'is', 'like', 'raisin', 'cookie', 'everyone', 'thinks', 'it', 'chocolate', 'chip', 'cookie', 'and', 'it', 'will', 'be', 'good', 'until', 'you', 'eat', 'it', 'and', 'realize', 'it', 'not', 'what', 'you', 'expected', 'unless', 'you', 'actually', 'like', 'raisin', 'cookies', 'story', 'the', 'story', 'is', 'somewhat', 'unoriginal', 'mainly', 'because', 'it', 'those', 'types', 'of', 'anime', 'where', 'you', 'are', 'embedded', 'certain', 'type', 'of', 'weapon', 'that', 'you', 'carry', 'on', 'for', 'the', 'rest', 'of', 'your', 'life', 'having', 'the', 'ability', 'to', 'summon', 'it', 'anytime', 'you', 'want', 'the', 'only', 'twist', 'is', 'you', 're', 'in', 'duo', 'now', 'every', 'time', 'you', 're', 'in', 'duo', 'it', 'likely', 'that', 'your', 'partner', 'is', 'someone', 'of', 'the', 'same', 'sex', 'right', 'well', 'in', 'this', 'anime', 'the', 'partner', 'is', 'girl', 'and', 'they', 'both', 'have', 'to', 'live

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'character', 'enjoyment', 'ever', 'trouble', 'spot', 'bottom', 'barrel', 'title', 'finding', 'product', 'studio', 'algorithm', 'intent', 'solely', 'product', 'show', 'much', 'integrity', 'use', 'car', 'salesman', 'tacky', 'poker', 'dot', 'tie', 'much', 'validity', 'mcdonald', 'promote', 'health', 'thinly', 'cut', 'apple', 'slice', 'bottle', 'process', 'milk', 'hogwash', 'pretend', 'artistic', 'merit', 'like', 'sift', 'bottom', 'tier', 'anime', 'much', 'let', 'occupy', 'time', 'medium', 'device', 'answer', 'quick', 'tip', 'help', 'typical', 'school', 'set', 'protagonist', 'pussy', 'magnet', 'beach', 'hot', 'spring', 'episode', 'love', 'interest', 'story', 'go', 'nowhere', 'unnecessary', 'fanservice', 'cover', 'photo', 'lead', 'easily', 'fluster', 'involve', 'female', 'go', 'tensionless', 'fight', 'school', 'rom', 'com', 'drop', 'hat', 'kind', 'pussy', 'beta', 'male', 'protagonist', 'unique', 'hax', 'superpower', 'lot', 'instance', 'lead', 'view', 'pervert', 'due', '

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 4), (14, 2), (15, 1), (16, 2), (17, 1), (18, 1), (19, 3), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 4), (31, 1), (32, 2), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 3), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 2), (93, 1), (94, 2), (95, 1), (96, 1), (97, 3), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 1), (105, 2), (106, 1), (107, 1), (108, 1), (109, 2), (110, 2),

In [21]:
# Using LDA to develop a topic model with 8 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 8 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.018*"show" + 0.015*"character" + 0.010*"make" + 0.009*"julie" + 0.009*"anime" + 0.008*"tor" + 0.007*"story" + 0.007*"girl" + 0.006*"really" + 0.005*"get"'), (1, '0.029*"anime" + 0.026*"character" + 0.023*"story" + 0.018*"harem" + 0.016*"watch" + 0.013*"overall" + 0.012*"enjoyment" + 0.012*"series" + 0.011*"go" + 0.011*"good"'), (2, '0.008*"title" + 0.005*"rito" + 0.004*"glass" + 0.004*"caitlin" + 0.003*"head" + 0.003*"sakuya" + 0.003*"defeat" + 0.003*"environment" + 0.002*"produce" + 0.002*"truly"'), (3, '0.025*"character" + 0.023*"show" + 0.017*"story" + 0.013*"much" + 0.012*"really" + 0.012*"anime" + 0.011*"overall" + 0.010*"plot" + 0.010*"feel" + 0.009*"get"'), (4, '0.009*"nao" + 0.008*"que" + 0.006*"para" + 0.003*"uma" + 0.003*"nada" + 0.003*"ruim" + 0.003*"mas" + 0.003*"muito" + 0.003*"drama" + 0.003*"spoiler"'), (5, '0.000*"character" + 0.000*"story" + 0.000*"show" + 0.000*"anime" + 0.000*"make" + 0.000*"get" + 0.000*"time" + 0.000*"good" + 0.000*"harem" + 0.000*"well"'),

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.163034  0.110277       1        1  36.816527
0      0.060806  0.096544       2        1  27.651995
1      0.184842 -0.071656       3        1  17.665012
3      0.147112 -0.097549       4        1  12.073803
6     -0.069521 -0.025896       5        1   2.522689
2     -0.184106 -0.004058       6        1   2.063850
4     -0.166291 -0.005190       7        1   1.197712
5     -0.135877 -0.002472       8        1   0.008411, topic_info=        Term        Freq       Total Category  logprob  loglift
13     anime  456.000000  456.000000  Default  30.0000  30.0000
203  overall  199.000000  199.000000  Default  29.0000  29.0000
255     show  267.000000  267.000000  Default  28.0000  28.0000
272    story  381.000000  381.000000  Default  27.0000  27.0000
319    watch  173.000000  173.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
122     girl    0.000514  142.077691   Topic8  -8.3042  -3.1457
375      end    0.000514  116.107181   Topic8  -8.3042  -2.9439
384    first    0.000514   83.761659   Topic8  -8.3043  -2.6174
319    watch    0.000514  173.379357   Topic8  -8.3043  -3.3449
173     main    0.000514  137.309010   Topic8  -8.3043  -3.1117

[557 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
5         1  0.163147  action
5         2  0.192810  action
5         3  0.444946  action
5         4  0.163147  action
5         5  0.029663  action
...     ...       ...     ...
328       2  0.210215   write
328       3  0.270277   write
328       4  0.450462   write
1240      2  0.864775   wrong
1240      5  0.108097   wrong

[761 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 2, 4, 7, 3, 5, 6])